###This uses the official nnunet implementation

Note: the implementation includes 5 fold cross validation built in -- throw everything into the training set

In [3]:
%pip install nnunetv2

                                              0.0/149.2 kB ? eta -:--:--
     -------------------------------------- 149.2/149.2 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel:

In [2]:
%pip install pytest-shutil

                                              0.0/320.9 kB ? eta -:--:--
     ---------------------------            235.5/320.9 kB 7.3 MB/s eta 0:00:01
     -------------------------------------- 320.9/320.9 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
torch.__version__

'2.1.0.dev20230602+cu121'

In [6]:
#grabbed from DatasetStatistics.ipynb

import glob
import SimpleITK as sitk

def hasMask(name):
    files = glob.glob(name+"/*.mha")

    reader = sitk.ImageFileReader()

    if(len(files)<2):
        return False
    return True


def axial(name):
    files = glob.glob(name+"/*.mha")
    if("ax" in files[0] and "ax" in files[1]):
        return True
    return False



CCF_count = 0

CCF_missing = [] #missing masks
CCF_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled/*'):
    
    if not (hasMask(name)):
        CCF_missing.append(name)
        continue
    if axial(name):
        CCF_count +=1
        CCF_list.append(name)
    

print("Total CCF: " + str(CCF_count))
print("Missing masks for CCF:" + str(CCF_missing))




UH_count = 0

UH_missing = [] #missing masks
UH_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled/*'):
    

    if not (hasMask(name)):
        UH_missing.append(name)
        continue
    if axial(name):
        UH_count +=1
        UH_list.append(name)

    

print("Total UH: " + str(UH_count))
print("Missing masks for UH:" + str(UH_missing))

print("Total: " +str(UH_count+CCF_count))
print("Expected: 166")



com_List = UH_list + CCF_list


Total CCF: 66
Missing masks for CCF:[]
Total UH: 100
Missing masks for UH:[]
Total: 166
Expected: 166


In [37]:
import pandas as pd
import numpy as np 

def removeNaN(x):
    temp = x[~pd.isnull(x)]
    return temp

split = pd.read_excel(r'G:\Shared drives\INVent\mfk58\New-Dataset-split.xlsx',sheet_name='splits')
split = split.to_numpy()

trset = removeNaN(split[:,0:3].flatten()) #first three columns
ext = removeNaN(split[:,3].flatten())



In [41]:
counts = [0,0,0]

import shutil



def copyToDirs(source: str, holdout: bool):
    if not holdout:
        images = r"G:\Shared drives\INVent\mfk58\trainingSet"
        labels = r"G:\Shared drives\INVent\mfk58\trainingLabels"
    else:
        images=r"G:\Shared drives\INVent\mfk58\holdoutSet"
        labels = r"G:\Shared drives\INVent\mfk58\holdoutLabels"
    
    files = glob.glob(dir+"/*.mha")
    label = ""
    img = ""
    if "label" in files[0]:
        label = files[0]
        img = files[1]
    else:
        label = files[1]
        img = files[0]
    
    shutil.copy2(label, labels)
    shutil.copy2(img, images)



for dir in com_List:
    files = glob.glob(dir+"/*.mha")
    label = ""
    img = ""
    if "label" in files[0]:
        label = files[0]
        img = files[1]
    else:
        label = files[1]
        img = files[0]
    
    s = img.replace(dir+"\\","") #should hopefully remove the portion that is not included in the actual dataset splits value
    s = s.replace(".mha","")

    if s in trset:
        counts[0] += 1
        copyToDirs(dir, False)
    elif s in ext:
        counts[1] += 1
        copyToDirs(dir, True)
    else:
        counts[2] += 1



In [50]:
import os

def rename(file: str, dir: str, label: bool):

    c = file
    s = ""

    if "CCF" in file:
        s = "CCF"
        c = c.replace("CCF_RectalCA","")
    elif "UH" in file:
        s = "UH"
        c = c.replace("UH_RectalCA","")
    else:
        raise Exception("Did not meet criteria for renaming")
    
    s = s + c[:4]

    
    if not label:
        s  = s + "_0000.mha"
    else:
        s = s + ".mha"
   
    os.rename(dir+"/"+file,dir+"/"+s)
    


In [56]:
import glob
images = r"G:\Shared drives\INVent\mfk58\trainingSet"
labels = r"G:\Shared drives\INVent\mfk58\trainingLabels"


e_images=r"G:\Shared drives\INVent\mfk58\holdoutSet"
e_labels = r"G:\Shared drives\INVent\mfk58\holdoutLabels"




In [68]:
import SimpleITK as sitk
def filterImage(img: str, val):
    r = sitk.ImageFileReader()
    r.SetFileName(img)
    a = r.Execute()
    arr = sitk.GetArrayFromImage(a)
    arr[arr != val] = 0
    changed = sitk.GetImageFromArray(arr)

    w = sitk.ImageFileWriter()
    w.SetFileName(img)
    changed.CopyInformation(a)
    w.Execute(changed)

e_labels = r"G:\Shared drives\INVent\mfk58\holdoutLabels"
labels = r"G:\Shared drives\INVent\mfk58\trainingLabels"
for file in glob.glob(e_labels+"/*.mha"):
    filterImage(file,1)
for file in glob.glob(labels+"/*.mha"):
    filterImage(file,1)

In [113]:
import glob
import SimpleITK as sitk
from scipy import ndimage
def center(arr):
    out = []
    arr[arr!=-1] = 1
    for dim in ndimage.center_of_mass(arr):
        out.append(round(dim))
    return out

def get_bounding_box(x):
    """ Calculates the bounding box of a ndarray"""
    mask = x == 0
    bbox = []
    all_axis = np.arange(x.ndim)
    for kdim in all_axis:
        nk_dim = np.delete(all_axis, kdim)
        mask_i = mask.all(axis=tuple(nk_dim))
        dmask_i = np.diff(mask_i)
        idx_i = np.nonzero(dmask_i)[0]

        low = min( idx_i )
        high = max( idx_i )
        s = slice(low + 1, high + 1)
        bbox.append(s)
    return bbox

def findCropBox(cen:tuple, img: tuple, crop:tuple):
    box = []
    for i in range(0,len(crop)):
        exp = (crop[i]-1)/2 # the distance in any one direction, again assumes odd size
        if ((exp+cen[i]) < img[i]) and (cen[i] - exp >= 0): #both directions within the inner bound -- perfect, move on
            box.append(slice(cen[i] - exp,exp+cen[i]+1)) #center the slice on the center
        else:
            #THE CROP TO SIZE SHOULD NOT EXCEED THE IMG SIZE, SO THIS SHOULDN'T BE AN ISSUE IN BOTH DIRECTIONS
            if not ((exp+cen[i]) < img[i]): #the outer boundary is expanded past
                box.append(slice(img[i]-crop[i],img[i])) #push directly against the edge of the image
            else: # went below 0
                box.append(slice(0,crop[i]))
    if(len(crop)<3):
        box.append(slice(0,img[2]))
    return tuple(box)

def Reverse(tuples): ##needed because the simple ITK reads in as z,y,x  - I am currently treating as x,y,z
    new_tup = tuples[::-1]
    return new_tup


#assumes that all of the dimensions in size are ODD
#should work for cropping in 2D (i.e., ignoring depth) and with 3D
def cropAll(size: tuple, img_dir: str, lbl_dir: str):
    imgs = glob.glob(img_dir+"\\*.mha")
    imgs.sort()
    masks = glob.glob(lbl_dir+"\\*.mha")
    masks.sort()
    assert len(imgs) == len(masks)

    r = sitk.ImageFileReader()
    for i in range(0,len(masks)):
        r.SetFileName(masks[i])
        a = sitk.GetArrayFromImage(r.Execute())
        bbx = get_bounding_box(a)
        
        arr = a[bbx[0].start:bbx[0].stop,bbx[1].start:bbx[1].stop,bbx[2].start:bbx[2].stop]
        c = center(arr)
        for i in range(0,len(c)):
            c[i] += bbx[i].start
        print(c)
        print(a.shape)
        print(Reverse(findCropBox(tuple(c),Reverse(a.shape),size)))












In [114]:
cropAll(tuple([145,135]),r"G:\Shared drives\INVent\mfk58\holdoutSet",r"G:\Shared drives\INVent\mfk58\holdoutLabels")


[99, 152, 104]
(198, 230, 230)
(slice(0, 198, None), slice(85.0, 220.0, None), slice(27.0, 172.0, None))
[57, 150, 111]
(84, 230, 230)
(slice(0, 84, None), slice(83.0, 218.0, None), slice(0, 145, None))
[96, 179, 152]
(216, 300, 300)
(slice(0, 216, None), slice(112.0, 247.0, None), slice(24.0, 169.0, None))
[162, 138, 140]
(279, 227, 290)
(slice(0, 279, None), slice(71.0, 206.0, None), slice(90.0, 235.0, None))
[132, 131, 112]
(187, 200, 200)
(slice(0, 187, None), slice(64.0, 199.0, None), slice(55, 200, None))
[129, 98, 93]
(180, 160, 160)
(slice(0, 180, None), slice(25, 160, None), slice(15, 160, None))
[117, 132, 102]
(286, 200, 200)
(slice(0, 286, None), slice(65.0, 200.0, None), slice(45.0, 190.0, None))
[77, 120, 102]
(168, 202, 202)
(slice(0, 168, None), slice(53.0, 188.0, None), slice(5.0, 150.0, None))
[59, 123, 77]
(120, 180, 180)
(slice(0, 120, None), slice(45, 180, None), slice(0, 145, None))
[86, 83, 77]
(140, 160, 160)
(slice(0, 140, None), slice(16.0, 151.0, None), slice